In [1]:
%matplotlib inline
import numpy as np
import os

In [2]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.34.0


# Connect Workspace

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: mltest
Azure region: eastus
Subscription id: 932c3e14-d0cf-4e41-9998-bdebb9bfa1cf
Resource group: rg-vast-mole


# Create experiment

In [4]:
from azureml.core import Experiment
experiment_name = 'myExp'

exp = Experiment(workspace=ws, name=experiment_name)

In [5]:
script_folder = './'
os.makedirs(script_folder, exist_ok=True)

with open(os.path.join(script_folder, 'myModel.py'), 'r') as f:
    print(f.read())

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



DATA_PATH="https://raw.githubusercontent.com/urtbest86/MLOps/master/result_train_dataset2.csv"
df = pd.read_csv(DATA_PATH)

DATA_PATH="https://raw.githubusercontent.com/urtbest86/MLOps/master/result_test_dataset2.csv"
test = pd.read_csv(DATA_PATH)

train=df.sample(frac=0.8)
val=df.sample(frac=0.2)

mean = train.mean(axis=0)
train -= mean
std = train.std(axis=0)
train /= std

test -= mean
test /= std

val-=mean
val/=std

train_data_set = train.values
x_train = train_data_set[:, 2:-1].astype(float)
y_train = train_data_set[:, -1].astype(float)

test_data_set = test.values
x_test = test_data_set[:, 2:-1].astype(float)
y_test = test_data_set[:, -1].astype(float)

val_data_set = val.values
x_val = val_data_set[:, 2:-1].astype(float)
y_val = val_data_set[:, -1].astype(float)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dens

# Attach existing compute resource

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "test-cluster")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")

found compute target: test-cluster


In [14]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.2

- pip:
  - h5py<=2.10.0
  - azureml-defaults
  - tensorflow-gpu==2.0.0
  - keras<=2.3.1
  - matplotlib
  - pandas
  - scikit-learn

Overwriting conda_dependencies.yml


In [15]:
from azureml.core import Environment

env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

'''# Specify a GPU base image
env.docker.enabled = True
env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'''

env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210806.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "keras-2.3.1",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "de

# 학습 실행 구성 및 제출
## ScriptRunConfig 만들기

In [39]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', 
        '--batch-size', 4,
        '--first-layer-neurons', 1024,
        '--second-layer-neurons', 500,
        '--third-layer-neurons', 300,
        '--forth-layer-neurons', 200,
        '--learning-rate', 0.001]

src = ScriptRunConfig(source_directory=script_folder,
                      script='myModel.py',
                      arguments=args,
                      compute_target=compute_name,
                      environment=env)

In [40]:
run = exp.submit(src)

In [41]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [42]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
myExp,myExp_1635338871_e58fa227,azureml.scriptrun,Running,Link to Azure Machine Learning studio,Link to Documentation


In [43]:
run.wait_for_completion(show_output=True)

RunId: myExp_1635338871_e58fa227
Web View: https://ml.azure.com/runs/myExp_1635338871_e58fa227?wsid=/subscriptions/932c3e14-d0cf-4e41-9998-bdebb9bfa1cf/resourcegroups/rg-vast-mole/workspaces/mltest&tid=19411b3a-b7ca-48d9-a1e2-31694d0ccb65

Streaming azureml-logs/55_azureml-execution-tvmps_fb0d01de94c4d0955f2187dc7ec102655bb7f0d215bb02998977b960d936ceea_d.txt

2021-10-27T12:48:49Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/mltest/azureml/myexp_1635338871_e58fa227/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/mltest/azureml/myexp_1635338871_e58fa227/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=21840 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/mltest/azureml/myexp_1635338871_e58fa227/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2021-10-27T12:48:50Z Successfully mounted a/an Blobfuse File System at /mnt/batch/tasks/shared/LS_root

{'runId': 'myExp_1635338871_e58fa227',
 'target': 'test-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-10-27T12:48:48.239904Z',
 'endTimeUtc': '2021-10-27T12:52:26.971882Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '8522029c-1a7c-4877-9e3c-79e12b0371ce',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'myModel.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--data-folder',
   '--batch-size',
   '4',
   '--first-layer-neurons',
   '1024',
   '--second-layer-neurons',
   '500',
   '--third-layer-neurons',
   '300',
   '--forth-layer-neurons',
   '200',
   '--learning-rate',
   '0.001'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'test-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'dat

In [44]:
print(run.get_metrics())

{'Final test loss(mse)': 0.4392291415821422, 'Final test loss(mae)': 0.5415523052215576, 'loss VS val_loss': 'aml://artifactId/ExperimentRun/dcid.myExp_1635338871_e58fa227/loss VS val_loss_1635339120.png'}


In [45]:
print(run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_fb0d01de94c4d0955f2187dc7ec102655bb7f0d215bb02998977b960d936ceea_d.txt', 'azureml-logs/65_job_prep-tvmps_fb0d01de94c4d0955f2187dc7ec102655bb7f0d215bb02998977b960d936ceea_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_fb0d01de94c4d0955f2187dc7ec102655bb7f0d215bb02998977b960d936ceea_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/95_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'loss VS val_loss_1635339120.png', 'outputs/Youjin_test_model.pkl']


In [46]:
'''model = run.register_model(model_name='myModelYoujin2', model_path='.')'''

"model = run.register_model(model_name='myModelYoujin2', model_path='.')"

In [47]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='Youjin-test', 
                           model_path='outputs/Youjin_test_model.pkl',
                           model_framework=Model.Framework.TENSORFLOW,
                           model_framework_version='2.0',
                           )

print(model.name, model.id, model.version, sep='\t')

Youjin-test	Youjin-test:2	2


In [48]:
'''os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)'''

"os.makedirs('./model', exist_ok=True)\n\nfor f in run.get_file_names():\n    if f.startswith('outputs/model'):\n        output_file_path = os.path.join('./model', f.split('/')[-1])\n        print('Downloading from {} to {} ...'.format(f, output_file_path))\n        run.download_file(name=f, output_file_path=output_file_path)"